# Import Packages

In [30]:
from PIL import Image

import os
import numpy as np
import matplotlib.pyplot as plt

# Load Images(Data)

In [35]:
def get_targets_and_image_dirs(dir):
    '''
    input
        dir : 이미지를 불러올 가장 상위 폴더의 경로를 입력합니다.
    ===================================================================
    return
        targets : 추후 label로 사용될 인물들의 명칭을 담은 리스트를 반환합니다.
        image_dirs : {label:key image_dir:value} 로 이뤄진 dictionary를 반환합니다.
    '''

    # 성별 폴더를 가져옵니다. (Male, Female)
    gender_folders = os.listdir(dir)
    
    # 각 성별의 인물 폴더를 담을 리스트(targets)와 이미지 경로가 담길 dictionary(image_dirs)를 선언합니다.
    targets, image_dirs = [], {}
    
    # 성별 폴더를 확인하면서, 각 폴더에 담긴 인물들의 리스트를 가져옵니다.
    for gender in gender_folders:
    
        # 현재 경로에 대한 정보를 변수에 담아둡니다.
        target_root = os.path.join(dir, gender)

        # target_folders에 찾은 인물들의 리스트를 담아둡니다.
        target_folders = os.listdir(target_root)

        # target 폴더별로 확인하면서 각 폴더별로 있는 이미지의 파일명을 가져옵니다.
        for target in target_folders:

            # 현재 경로에 대한 정보를 변수에 담아둡니다.
            image_root = os.path.join(target_root, target)

            # image_names에 찾은 파일명들의 리스트를 담아둡니다.
            image_names = os.listdir(image_root)

            # image_dirs의 target key를 선언해줍니다.
            image_dirs[target] = []

            for image_name in image_names:
                # 이미지의 경로를 초기화합니다.
                image_dir = os.path.join(image_root, image_name)

                # return할 image_dirs폴더에 각 target에 맞는 이미지의 경로를 담아줍니다.
                image_dirs[target].append(image_dir)

        # 미리 생성한 targets에 인물들의 리스트를 함께 담아줍니다.
        targets.extend(target_folders)

    return targets, image_dirs


# 이미지 경로를 통해 실제 이미지를 불러오고, 해당 이미지에 맞는 label을 만들어서 반환합니다.
def load_images_and_set_labels(targets, image_dirs):
    '''
    input
        targets : 이미지에 맞는 label에 사용될 list를 입력합니다.
        image_dirs : 불러올 이미지의 경로가 담긴 list를 입력합니다.
    ===================================================================
    return
        images : image_dir로 불러온 np.ndarray타입의 이미지를 담은 리스트를 반환합니다.
        labels : images에 같은 index에 위치한 이미지의 labeld을 담은 리스트를 반환합니다.
    '''
    # load된 image를 담을 리스트와 대응하는 label을 담을 리스트를 선언합니다.
    images, labels = [], []

    # target값을 image_dirs의 key로 활용합니다.
    for target in targets:
        # target을 key로하는 value들을 불러옵니다.
        for image_dir in image_dirs[target]:
            # 해당 이미지에 맞는 label을 추가합니다.
            labels.append(target)

            # Image 타입의 이미지를 불러옵니다. Type = PIL.Image
            image = Image.open(image_dir)

            # 이미지를 numpy.ndarray 타입으로 변경합니다.
            np_image = np.array(image)

            # images에 image를 추가합니다.
            images.append(np_image)

    return images, labels

In [36]:
# 본폴더에 있는 data folder로 경로를 설정한다.
dir = os.path.join(os.getcwd(), 'data')

targets, image_dirs = get_targets_and_image_dirs(dir)
images, labels = load_images_and_set_labels(targets, image_dirs)

In [38]:
for image, label in zip(images, labels):
    print(type(image), label)

<class 'numpy.ndarray'> IU
<class 'numpy.ndarray'> IU
<class 'numpy.ndarray'> IU
<class 'numpy.ndarray'> IU
<class 'numpy.ndarray'> IU
<class 'numpy.ndarray'> JeongHoYeon
<class 'numpy.ndarray'> JeongHoYeon
<class 'numpy.ndarray'> JeongHoYeon
<class 'numpy.ndarray'> JeongHoYeon
<class 'numpy.ndarray'> JeongHoYeon
<class 'numpy.ndarray'> JeongYeon
<class 'numpy.ndarray'> JeongYeon
<class 'numpy.ndarray'> JeongYeon
<class 'numpy.ndarray'> JeongYeon
<class 'numpy.ndarray'> JeongYeon
<class 'numpy.ndarray'> LeeYuBi
<class 'numpy.ndarray'> LeeYuBi
<class 'numpy.ndarray'> LeeYuBi
<class 'numpy.ndarray'> LeeYuBi
<class 'numpy.ndarray'> LeeYuBi
<class 'numpy.ndarray'> YunHa
<class 'numpy.ndarray'> YunHa
<class 'numpy.ndarray'> YunHa
<class 'numpy.ndarray'> YunHa
<class 'numpy.ndarray'> YunHa
<class 'numpy.ndarray'> GongYu
<class 'numpy.ndarray'> GongYu
<class 'numpy.ndarray'> GongYu
<class 'numpy.ndarray'> GongYu
<class 'numpy.ndarray'> GongYu
<class 'numpy.ndarray'> HaJungWoo
<class 'numpy.nd

In [28]:
image_dirs

{'IU': ['c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\IU\\0.jpg',
  'c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\IU\\14.jpg',
  'c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\IU\\18.jpg',
  'c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\IU\\2.jpg',
  'c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\IU\\7.jpg'],
 'JeongHoYeon': ['c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\JeongHoYeon\\14.jpg',
  'c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\JeongHoYeon\\2.jpg',
  'c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\JeongHoYeon\\4.jpg',
  'c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\JeongHoYeon\\7.jpg',
  'c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\JeongHoYeon\\9.jpg'],
 'JeongYeon': ['c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\JeongYeon\\0.jpg',
  'c:\\DJ_develop\\Python\\Data_Augmentation\\data\\Female\\JeongYeon\\10.jpg',
  'c:\\DJ_develop\\Python\\Data_Augme

In [ ]:
class Image_Augmentation():
    def __init__(self, image_list, save_dir):
        self.image_list = image_list

    # Crop 기능 수행
    def crop(self, random=True):
        pass

    # Resize 기능 수행
    def resize(self, random=True):
        pass

    # Flip 기능 수행
    def flip(self, random=True):
        pass

    # Rotate 기능 수행
    def rotate(self, random=True):
        pass

    # Random Erase 기능 수헹
    def random_erase(self):
        pass

    # 처리 전, 후 이미지 확인 기능 수행
    def show_images(self, images):
        pass